# Scrape Website Content

This Notebook can be run to scrape a hierarchical html website and place the raw output into Azure blob storage.

## Install dependencies
First we install the dependencies required in ths notebook.

In [ ]:
# Install dependencies

%pip install python-dotenv
%pip install BeautifulSoup4

## Load credentials
Next we load the environment variables needed by the following cells from the `.env` file.

In [ ]:
# Load credentials
import os
from dotenv import load_dotenv
load_dotenv()

## Set execution parameters
Set the URL of the website to be scraped.

In [ ]:
# Define the URL of the website to scrape
url = 'https://quotes.toscrape.com/'

# Azure Blob Storage connection string and container name
connection_string = os.getenv("BLOB_CONNECTION_STRING")
container_name = os.getenv("BLOB_CONTAINER_NAME")

print(f"Connection string: {connection_string}")

## Perform the scrape
Now we perform the scrape against the website.

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Create BlobServiceClient and container client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Ensure the container exists
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists or could not be created: {e}")

# Function to upload a file to Azure Blob Storage
def upload_to_azure(file_name, data):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
    blob_client.upload_blob(data)
    print(f"Uploaded {file_name} to Azure Blob Storage.")

# Recursive function to scrape hierarchical website and save pages
def scrape_and_save(url, base_url, visited=set()):
    if url in visited:
        return

    visited.add(url)
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve {url}")
        return

    # Parse HTML with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Save HTML content to Azure Blob Storage
    file_name = url.replace(base_url, '').strip('/').replace('/', '_') + '.html'
    if not file_name:
        file_name = 'index.html'
    upload_to_azure(file_name, response.text)

    # Find all links and recurse
    for link in soup.find_all('a', href=True):
        link_url = link['href']
        # Handle relative URLs
        if not link_url.startswith('http'):
            link_url = base_url + link_url.strip('/')
        # Recurse on the found link
        scrape_and_save(link_url, base_url, visited)

# Example usage
base_url = 'https://example.com'  # Replace with the root of the website you want to scrape
start_url = base_url  # The starting point of the website
scrape_and_save(start_url, base_url)
